In [ ]:
# 📘 Proyecto de Regresión Profesional con Keras, GridSearchCV y SHAP

# ============================
# 1. Carga y preprocesamiento
# ============================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Cargar dataset
boston = load_boston()
X = boston.data
y = boston.target

# División y escalado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ============================
# 2. Definición del modelo
# ============================

from tensorflow import keras
from keras import layers, Input, regularizers

def build_model(n_hidden=2, n_neurons=64, learning_rate=0.01, dropout_rate=0.0, l2_reg=0.0):
    input_layer = Input(shape=(X_train_scaled.shape[1],))
    x = input_layer
    for _ in range(n_hidden):
        x = layers.Dense(n_neurons, activation='relu',
                         kernel_regularizer=regularizers.l2(l2_reg))(x)
        if dropout_rate > 0:
            x = layers.Dropout(dropout_rate)(x)
    output = layers.Dense(1)(x)

    model = keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse',
                  metrics=['mae'])
    return model

# ============================
# 3. GridSearchCV
# ============================

from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

keras_reg = KerasRegressor(model=build_model, verbose=0)

param_grid = {
    "model__n_hidden": [1, 2],
    "model__n_neurons": [32, 64],
    "model__learning_rate": [0.001, 0.01],
    "model__dropout_rate": [0.0, 0.2],
    "model__l2_reg": [0.0, 0.01],
    "batch_size": [16],
    "epochs": [200]
}

grid_search = GridSearchCV(keras_reg, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

print("\nMejores hiperparámetros:", grid_search.best_params_)

# ============================
# 4. Entrenamiento final y evaluación
# ============================

best_model = grid_search.best_estimator_.model_
history = best_model.fit(X_train_scaled, y_train,
                         validation_data=(X_test_scaled, y_test),
                         epochs=300,
                         callbacks=[keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)],
                         verbose=0)

def plot_learning_curves(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['mae'], label='Train MAE')
    plt.plot(history.history['val_mae'], label='Val MAE')
    plt.title('Error Absoluto Medio (MAE)')
    plt.grid(); plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train MSE')
    plt.plot(history.history['val_loss'], label='Val MSE')
    plt.title('Error Cuadrático Medio (MSE)')
    plt.grid(); plt.legend()
    plt.show()

plot_learning_curves(history)

# ============================
# 5. Evaluación en test
# ============================

predictions = best_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)
print(f"\nRMSE en test: {rmse:.2f}")
print(f"R^2 en test: {r2:.2f}")

# ============================
# 6. Interpretación con SHAP
# ============================

import shap

explainer = shap.Explainer(best_model.predict, X_train_scaled[:100])
shap_values = explainer(X_test_scaled[:10])

# Gráfico local (1 muestra)
shap.plots.waterfall(shap_values[0])

# Importancia global
shap.plots.beeswarm(shap_values)

# ============================
# 7. Posibles mejoras (nivel master)
# ============================

# - Probar modelos de ensamble (e.g., stacking con XGBoost)
# - Validación cruzada más robusta (Repeated K-Fold)
# - Automatización de hiperparámetros con Optuna o Keras Tuner
# - Feature selection con SHAP + retrain
# - EarlyStopping monitorizando 'val_mae' o custom callback
# - Logging con TensorBoard para análisis detallado
